Old Analysis of operator domain, with different approach and different bio tools

# Operator domain data analysis
- Tables 3 and 4 in the paper number of tools used in the workflows 
- (e.g., bioinformatics tools such as samtools, bcftools)

In this seperate notebook, we want to analyze the operators in the nf-core git-repos. 
The target operators are provided in two excel sheets and can be found in the folder 'data'. Also we want to consider the operators from the Nexflow documentation here: https://www.nextflow.io/docs/latest/operator.html

In [ ]:
import pandas as pd

df_operators = pd.read_excel('./data/all_operator_dataset.xlsx')
#df_bioinformatic_tools = pd.read_excel('./data/bioinformatics_tools.xlsx')

pattern_list = df_operators['operator'].tolist()

print(pattern_list)

In [ ]:
len(pattern_list)

In [ ]:
df_operators_clustured = pd.read_csv('./data/operator_dataset_clustered.csv')

We have 6 unique domain groups: ["Bioinformatics", "Control flow", "File processing", "File system", "Code execution", "Data handling", "Misc", "Noise", "under 10"]


e.g.: Assigned Group 0 = "Bioinformatics"
if domain >= 6: "Other" domain

In [ ]:
df_operators_clustured.head()

In [ ]:
df_operators_clustured['Assigned Group'].unique()

In [ ]:
df

# Operator domain data (all operators from csv) in processes

In [ ]:
import os
import re
import csv
import pandas as pd

def find_operators_in_processes(directory, pattern_list, output_file_name):
    result_list = []

    for pipeline in os.listdir(directory):
        subdirectory_path = os.path.join(directory, pipeline)
        folder_name = os.path.basename(subdirectory_path)

        for root, dirs, files in os.walk(subdirectory_path):
            for filename in files:
                file_path = os.path.join(root, filename)
                
                if filename.endswith('.nf') or filename.endswith('.config'):
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                        lines = file.readlines()
                    
                    process_flag = False
                    process_name = None
                    brace_stack = []

                    process_pattern = r'process\s+(\w+)\s*{'

                    for line_num, line in enumerate(lines):
                        stripped_line = line.strip()
                        
                        if line.strip().startswith(('#', '//', '*')):
                                continue

                        if re.search(process_pattern, stripped_line):
                            process_flag = True
                            process_match = re.search(process_pattern, stripped_line)
                            if process_match:
                                process_name = process_match.group(1)
                            print(f'Process {process_name} in filepath {file_path}')
                            brace_stack.append('{')

                        if '{' in stripped_line:
                            brace_stack.append('{')

                        if '}' in stripped_line:
                            if brace_stack:
                                brace_stack.pop()
                                if not brace_stack and process_flag:
                                    process_flag = False

                        if process_flag:
                            pattern_found = False
                            for pattern in pattern_list:

                                regex_pattern = re.compile(re.escape(pattern))
                                #regex_pattern = re.compile(r'(\.\s*|\s+)' + re.escape(pattern) + r'(\s+|\.\s*)')
                                if regex_pattern.search(stripped_line):
                                    result_list.append([folder_name, process_name, pattern, file_path, line_num+1, stripped_line])
                                    print(f'Operator found {pattern} with content: {stripped_line}')
                                    pattern_found = True

                    if brace_stack:
                        print("Error: Unclosed process block in file:", file_path)

    with open(f'./results/{output_file_name}.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['pipeline', 'process_name', 'operator', 'file_path', 'line_number', 'line_content'])
        writer.writerows(result_list)

In [ ]:
directory_path = './git_repos'

df_operators = pd.read_excel('./data/all_operator_dataset.xlsx')
operator_list = [str(operator) for operator in df_operators['operator']]

output_file_name = 'all_operators_in_processes'

find_operators_in_processes(directory_path, operator_list, output_file_name)

Bioinformatic tools in processes

In [ ]:
df_operators = pd.read_excel('./data/bioinformatics_tools.xlsx')
pattern_list = [str(pattern) for pattern in df_operators['operator']]

output_file_name = 'all_operators_in_processes'

find_operators_in_processes(directory_path, pattern_list, output_file_name)

# Parser for operator domain data occurences in config files

In [12]:
import os
import re
import csv
import pandas as pd

def find_operators_in_config(directory, pattern_list, output_file_name):
    result_list = []

    for pipeline in os.listdir(directory):
        subdirectory_path = os.path.join(directory, pipeline)
        folder_name = os.path.basename(subdirectory_path)

        for root, dirs, files in os.walk(subdirectory_path):
            for filename in files:
                file_path = os.path.join(root, filename)
                
                if filename.endswith('.config'):
                #if filename.endswith('.nf'):
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                        lines = file.readlines()

                    for line_num, line in enumerate(lines):
                        stripped_line = line.strip()
                        
                        if line.strip().startswith(('#', '//', '*')):
                            continue
                        
                        for pattern in pattern_list:

                            regex_pattern = re.compile(re.escape(pattern))
                            if regex_pattern.search(stripped_line):
                                result_list.append([folder_name, pattern, file_path, line_num+1, stripped_line])
                                print(f'Operator found {pattern} with content: {stripped_line}')

    with open(f'./results/{output_file_name}.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['pipeline', 'operator', 'file_path', 'line_number', 'line_content'])
        writer.writerows(result_list)

In [13]:
'''directory_path = './git_repos'

df_operators = pd.read_excel('./data/all_operator_dataset.xlsx')
operator_list = [str(pattern) for pattern in df_operators['operator']]

output_file_name = 'all_operators_in_config'

find_operators_in_config(directory_path, operator_list, output_file_name)'''

"directory_path = './git_repos'\n\ndf_operators = pd.read_excel('./data/all_operator_dataset.xlsx')\noperator_list = [str(pattern) for pattern in df_operators['operator']]\n\noutput_file_name = 'all_operators_in_config'\n\nfind_operators_in_config(directory_path, operator_list, output_file_name)"

Bioinformatic tools

In [14]:
directory_path = './git_repos'

df_operators = pd.read_excel('./data/bioinformatics_tools.xlsx')
operator_list = [str(pattern) for pattern in df_operators['operator']]

output_file_name = 'bioinformatic_tools_in_config'

find_operators_in_config(directory_path, operator_list, output_file_name)

Operator found multiqc with content: skip_multiqc               = false
Operator found multiqc with content: multiqc_config             = null
Operator found multiqc with content: multiqc_title              = null
Operator found multiqc with content: multiqc_logo               = null
Operator found multiqc with content: max_multiqc_email_size     = '25.MB'
Operator found multiqc with content: multiqc_methods_description = null
Operator found fastp with content: path: { "${params.outdir}/fastp/${meta.id}" },
Operator found fastp with content: path: { "${params.outdir}/fastp/${meta.id}/" },
Operator found fastp with content: pattern: "*.fastp.fastq.gz"
Operator found fastqc with content: path: { "${params.outdir}/fastqc/postassembly" },
Operator found multiqc with content: ext.args   = params.multiqc_title ? "--title \"$params.multiqc_title\"" : ''
Operator found multiqc with content: path: { "${params.outdir}/multiqc" },
Operator found cutadapt with content: cutadapt_min_overlap = 3
Ope

Operator found qiime with content: nextflow run nf-core/ampliseq -profile test_qiimecustom,<docker/singularity> --outdir <OUTDIR>
Operator found qiime with content: config_profile_description = 'Minimal test dataset to check --qiime_ref_tax_custom'
Operator found qiime with content: qiime_ref_tax_custom = "https://raw.githubusercontent.com/nf-core/test-datasets/ampliseq/testdata/85_greengenes.fna.gz,https://raw.githubusercontent.com/nf-core/test-datasets/ampliseq/testdata/85_greengenes.tax.gz"
Operator found qiime with content: skip_qiime_downstream = true
Operator found kraken2 with content: kraken2_ref_tax_custom = "https://genome-idx.s3.amazonaws.com/kraken/16S_Greengenes13.5_20200326.tgz"
Operator found kraken2 with content: kraken2_assign_taxlevels = "D,P,C,O"
Operator found qiime with content: qiime_ref_tax_custom = "https://raw.githubusercontent.com/nf-core/test-datasets/ampliseq/testdata/85_greengenes.tar.gz"
Operator found qiime with content: skip_qiime_downstream = true
Opera

Operator found samtools with content: path: { "${params.outdir}/${params.aligner}/library/samtools_stats" },
Operator found bwa with content: if (params.aligner == 'bwa') {
Operator found bwa with content: params.bwa_min_score ? " -T ${params.bwa_min_score}" : '',
Operator found bowtie2 with content: if (params.aligner == 'bowtie2') {
Operator found picard with content: path: { "${params.outdir}/${params.aligner}/merged_library/picard_metrics" },
Operator found samtools with content: path: { "${params.outdir}/${params.aligner}/merged_library/samtools_stats" },
Operator found samtools with content: path: { "${params.outdir}/${params.aligner}/merged_library/samtools_stats" },
Operator found samtools with content: path: { "${params.outdir}/${params.aligner}/merged_library/samtools_stats" },
Operator found picard with content: if (!params.skip_picard_metrics) {
Operator found picard with content: path: { "${params.outdir}/${params.aligner}/merged_library/picard_metrics" },
Operator found p

Operator found samtools with content: samtools_collate_fast = true
Operator found fastqc with content: skip_initial_fastqc = false
Operator found fastqc with content: skip_trimming_fastqc = false
Operator found samtools with content: skip_samtools_stats = false
Operator found multiqc with content: multiqc_config = false
Operator found multiqc with content: max_multiqc_email_size = 25.MB
Operator found bwa with content: bwa         = "${params.igenomes_base}/Homo_sapiens/Ensembl/GRCh37/Sequence/BWAIndex/genome.fa"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Homo_sapiens/Ensembl/GRCh37/Sequence/Bowtie2Index/"
Operator found bismark with content: bismark     = "${params.igenomes_base}/Homo_sapiens/Ensembl/GRCh37/Sequence/BismarkIndex/"
Operator found bwa with content: bwa         = "${params.igenomes_base}/Homo_sapiens/NCBI/GRCh38/Sequence/BWAIndex/genome.fa"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Homo_sapiens/NCBI/GRCh3

Operator found bwa with content: bwa         = "${params.igenomes_base}/Glycine_max/Ensembl/Gm01/Sequence/BWAIndex/version0.6.0/"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Glycine_max/Ensembl/Gm01/Sequence/Bowtie2Index/"
Operator found bismark with content: bismark     = "${params.igenomes_base}/Glycine_max/Ensembl/Gm01/Sequence/BismarkIndex/"
Operator found bwa with content: bwa         = "${params.igenomes_base}/Macaca_mulatta/Ensembl/Mmul_1/Sequence/BWAIndex/version0.6.0/"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Macaca_mulatta/Ensembl/Mmul_1/Sequence/Bowtie2Index/"
Operator found bismark with content: bismark     = "${params.igenomes_base}/Macaca_mulatta/Ensembl/Mmul_1/Sequence/BismarkIndex/"
Operator found bwa with content: bwa         = "${params.igenomes_base}/Oryza_sativa_japonica/Ensembl/IRGSP-1.0/Sequence/BWAIndex/version0.6.0/"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Oryz

...

Operator found bwa with content: bwa         = "${params.igenomes_base}/Saccharomyces_cerevisiae/UCSC/sacCer3/Sequence/BWAIndex/genome.fa"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Saccharomyces_cerevisiae/UCSC/sacCer3/Sequence/Bowtie2Index/"
Operator found bismark with content: bismark     = "${params.igenomes_base}/Saccharomyces_cerevisiae/UCSC/sacCer3/Sequence/BismarkIndex/"
Operator found bwa with content: bwa         = "${params.igenomes_base}/Sus_scrofa/UCSC/susScr3/Sequence/BWAIndex/genome.fa"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Sus_scrofa/UCSC/susScr3/Sequence/Bowtie2Index/"
Operator found bismark with content: bismark     = "${params.igenomes_base}/Sus_scrofa/UCSC/susScr3/Sequence/BismarkIndex/"
Operator found fastqc with content: 'fastqc' {
Operator found multiqc with content: 'multiqc' {
Operator found picard with content: 'picard/sortsam_parsed' {
Operator found picard with content: 'picard/sortsam_co

Operator found bwa with content: bwa         = "${params.igenomes_base}/Canis_familiaris/UCSC/canFam3/Sequence/BWAIndex/version0.6.0/"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Canis_familiaris/UCSC/canFam3/Sequence/Bowtie2Index/"
Operator found bismark with content: bismark     = "${params.igenomes_base}/Canis_familiaris/UCSC/canFam3/Sequence/BismarkIndex/"
Operator found bwa with content: bwa         = "${params.igenomes_base}/Danio_rerio/UCSC/danRer10/Sequence/BWAIndex/version0.6.0/"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Danio_rerio/UCSC/danRer10/Sequence/Bowtie2Index/"
Operator found bismark with content: bismark     = "${params.igenomes_base}/Danio_rerio/UCSC/danRer10/Sequence/BismarkIndex/"
Operator found bwa with content: bwa         = "${params.igenomes_base}/Drosophila_melanogaster/UCSC/dm6/Sequence/BWAIndex/version0.6.0/"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Drosophi

Operator found kraken2 with content: run_kraken2                           = false
Operator found kraken2 with content: run_kraken2                           = false
Operator found kraken2 with content: run_kraken2                           = true
Operator found kraken2 with content: run_kraken2                           = false
Operator found kraken2 with content: run_kraken2                           = false
Operator found kraken2 with content: run_kraken2                           = true
Operator found multiqc with content: multiqc_config             = null
Operator found multiqc with content: multiqc_title              = null
Operator found multiqc with content: multiqc_logo               = null
Operator found multiqc with content: max_multiqc_email_size     = '25.MB'
Operator found multiqc with content: multiqc_methods_description = null
Operator found bwa with content: bwa         = "${params.igenomes_base}/Homo_sapiens/Ensembl/GRCh37/Sequence/BWAIndex/version0.6.0/"
Operator fou

Operator found multiqc with content: multiqc_config             = null
Operator found multiqc with content: multiqc_title              = null
Operator found multiqc with content: multiqc_logo               = null
Operator found multiqc with content: max_multiqc_email_size     = '25.MB'
Operator found multiqc with content: multiqc_methods_description = null
Operator found bwa with content: bwa         = "${params.igenomes_base}/Homo_sapiens/Ensembl/GRCh37/Sequence/BWAIndex/version0.6.0/"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Homo_sapiens/Ensembl/GRCh37/Sequence/Bowtie2Index/"
Operator found bismark with content: bismark     = "${params.igenomes_base}/Homo_sapiens/Ensembl/GRCh37/Sequence/BismarkIndex/"
Operator found bwa with content: bwa         = "${params.igenomes_base}/Homo_sapiens/NCBI/GRCh38/Sequence/BWAIndex/version0.6.0/"
Operator found bowtie2 with content: bowtie2     = "${params.igenomes_base}/Homo_sapiens/NCBI/GRCh38/Sequence/Bowtie2Index

Operator found bcftools with content: if (!variant_caller) { variant_caller = params.protocol == 'amplicon' ? 'ivar' : 'bcftools' }
Operator found fastqc with content: if (!params.skip_fastqc) {
Operator found fastqc with content: path: { "${params.outdir}/fastqc/raw" },
Operator found fastp with content: if (!params.skip_fastp) {
Operator found fastp with content: path: { "${params.outdir}/fastp" },
Operator found fastp with content: path: { "${params.outdir}/fastp/log" },
Operator found fastp with content: path: { "${params.outdir}/fastp" },
Operator found fastqc with content: if (!params.skip_fastqc) {
Operator found fastqc with content: path: { "${params.outdir}/fastqc/trim" },
Operator found kraken2 with content: if (!params.skip_kraken2) {
Operator found kraken2 with content: path: { "${params.outdir}/kraken2" },
Operator found bowtie2 with content: path: { "${params.outdir}/variants/bowtie2/log" },
Operator found bowtie2 with content: path: { "${params.outdir}/variants/bowtie2/u

# Bioinformatic tools in configuration files

In [15]:
import pandas as pd
import numpy as np

df_bioinformatics = pd.read_csv('./results/bioinformatic_tools_in_config.csv')
df_bioinformatics

,pipeline,operator,file_path,line_number,line_content
0,airrflow,multiqc,./git_repos\airrflow\nextflow.config,107,skip_multiqc = false
1,airrflow,multiqc,./git_repos\airrflow\nextflow.config,108,multiqc_config = null
2,airrflow,multiqc,./git_repos\airrflow\nextflow.config,109,multiqc_title = null
3,airrflow,multiqc,./git_repos\airrflow\nextflow.config,110,multiqc_logo = null
4,airrflow,multiqc,./git_repos\airrflow\nextflow.config,111,max_multiqc_email_size = '25.MB'
...,...,...,...,...,...
8299,viralrecon,multiqc,./git_repos\viralrecon\conf\modules_nanopore.c...,345,"path: { ""${params.outdir}/multiqc/${params.art..."
8300,viralrecon,kraken2,./git_repos\viralrecon\conf\test.config,31,kraken2_db = 'https://raw.githubusercontent.co...
8301,viralrecon,bcftools,./git_repos\viralrecon\conf\test_full_sispa.co...,26,variant_caller = 'bcftools'
8302,viralrecon,kraken2,./git_repos\viralrecon\conf\test_sispa.config,29,kraken2_db = 'https://raw.githubusercontent.co...


In [16]:
pipelines_total = df_bioinformatics.groupby('pipeline').size().reset_index(name='bioinformatic_occur')
config_count = df_bioinformatics.groupby('file_path').size().reset_index(name='bioinformatic_occur')

print(f'Matched in {len(pipelines_total)} pipelines and {len(config_count)} config files')

Matched in 88 pipelines and 332 config files


In [17]:
df_bioinformatics

,pipeline,operator,file_path,line_number,line_content
0,airrflow,multiqc,./git_repos\airrflow\nextflow.config,107,skip_multiqc = false
1,airrflow,multiqc,./git_repos\airrflow\nextflow.config,108,multiqc_config = null
2,airrflow,multiqc,./git_repos\airrflow\nextflow.config,109,multiqc_title = null
3,airrflow,multiqc,./git_repos\airrflow\nextflow.config,110,multiqc_logo = null
4,airrflow,multiqc,./git_repos\airrflow\nextflow.config,111,max_multiqc_email_size = '25.MB'
...,...,...,...,...,...
8299,viralrecon,multiqc,./git_repos\viralrecon\conf\modules_nanopore.c...,345,"path: { ""${params.outdir}/multiqc/${params.art..."
8300,viralrecon,kraken2,./git_repos\viralrecon\conf\test.config,31,kraken2_db = 'https://raw.githubusercontent.co...
8301,viralrecon,bcftools,./git_repos\viralrecon\conf\test_full_sispa.co...,26,variant_caller = 'bcftools'
8302,viralrecon,kraken2,./git_repos\viralrecon\conf\test_sispa.config,29,kraken2_db = 'https://raw.githubusercontent.co...


In [18]:
bio_df = df_bioinformatics.groupby('operator').agg({'pipeline': 'nunique', 'file_path': 'nunique'}).reset_index().sort_values(by='pipeline', ascending=False)
#grouped_df = merged_df.groupby('Assigned Group').agg({'config_file': 'nunique', 'operator_in_file': 'nunique'}).reset_index()

In [19]:
bio_df.head()

,operator,pipeline,file_path
23,multiqc,83,140
5,bowtie2,63,84
6,bwa,63,106
4,bismark,57,59
11,fastqc,40,62


# Operator domain data analysis

- we get all the lines with occuring operators, filter by length of operator due to noisy data!

In [1]:
import pandas as pd

df = pd.read_csv('./results/all_operators_in_config.csv')
df_operator_clustered = pd.read_csv('./data/operator_dataset_clustered.csv')

In [2]:
df = pd.read_csv('./results/all_operators_in_config.csv')

Here we filter by the length of the operator, due to noise in data results similar approach to GitLab Snakemake (https://gitlab.informatik.hu-berlin.de/pohlseba/github_scraping/-/blob/main/analysis/operators/operators.py, line 222)

In [3]:
df = df[df['operator'].str.len() > 3]

By applying that, we can make sure that every operator occures just once in a pipline and we dont count multiple occurences

In [4]:
# we get each operator occurence once per file from 76137 to 12132  entries

df = df.groupby(['pipeline', 'operator', 'file_path']).size().reset_index(name='operator_in_file')

In [5]:
df = df.rename(columns={'operator': 'Operator', 'file_path': 'config_file'})

In [6]:
merged_df = pd.merge(df, df_operator_clustered, on='Operator')

In [7]:
# 1022 configuration files
merged_df['config_file'].nunique()

1022

Group by Assigned Group

In [8]:
# n = number of operators to be shown
def most_common_operators(x, n=4):
    return list(x.value_counts().head(n).index)

In [15]:
agg_bio = merged_df.groupby('Assigned Group').agg({'pipeline': 'nunique', 'config_file': 'nunique', 'operator_in_file': 'count', 'Operator': lambda x: most_common_operators(x)}).reset_index()
agg_bio

,Assigned Group,pipeline,config_file,operator_in_file,Operator
0,0,87,406,680,"[fasta, fastq, fastqc, fastq.gz]"
1,1,94,446,580,"[docker, else, centrifuge, classifier]"
2,2,94,838,1113,"[params, print, echo, paired]"
3,3,13,22,22,"[zcat, coverm, cpan, sourmash]"
4,4,94,715,2060,"[script, singularity, report, conda]"
5,5,7,11,20,"[gzip, bgzip, gunzip, unzip]"
6,6,94,1013,7676,"[file, test, time, input]"


In [16]:
plot = merged_df.groupby('Assigned Group', 'config_file').agg({'operator_in_file': 'nunique'}).reset_index()
plot

ValueError: No axis named config_file for object type DataFrame

- Assigned Group 0:    bioinformatics tool
- Assigned Group 1:    control flow
- Assigned Group 2:    file processing
- Assigned Group 3:    file system
- Assigned Group 4:    code execution
- Assigned Group 5:    data handling
- Assigned Group 6:    other

# Analysis of operator domain data in processes

In [71]:
import pandas as pd

df = pd.read_csv('./results/all_operators_in_processes.csv')

df_operator_clustered = pd.read_csv('./data/operator_dataset_clustered.csv')

In [72]:
df = df[df['operator'].str.len() > 3]

In [73]:
df = df.rename(columns={'operator': 'Operator'})

In [74]:
merged_df = pd.merge(df, df_operator_clustered, on='Operator')

In [75]:
grouped_df = merged_df.groupby('Assigned Group').agg({'pipeline': 'nunique', 'process_name': 'nunique', 'Operator': 'count', 'file_path': 'nunique'}).reset_index()

Add ratio = # occuring processes /# total rules

In [76]:
sum_processes = merged_df['process_name'].nunique()

In [77]:
sum_processes

1982

In [78]:
grouped_df['ratio'] = grouped_df['process_name']/sum_processes

In [79]:
grouped_df

,Assigned Group,pipeline,process_name,Operator,file_path,ratio
0,0,88,863,9315,1269,0.435419
1,1,93,1588,4045,2463,0.801211
2,2,94,1622,8498,1943,0.818365
3,3,36,35,120,42,0.017659
4,4,94,1979,18899,2573,0.998486
5,5,70,187,1120,283,0.094349
6,6,94,1982,33788,2576,1.000000
